# Phase 3: Analysis & Machine Learning (Microsoft Corp)

## Objective
In this phase, we connect to the processed SQL database, perform Exploratory Data Analysis (EDA) on the financial fundamentals, and build a Machine Learning model to forecast future Earnings Per Share (EPS).

**Note:** Due to API access limitations on Daily Price Candles, this analysis focuses on **Quarterly Fundamental Data** (Sales Growth, EPS, Net Margin), which provides a robust view of the company's financial health and trajectory.

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Database Connection
DB_PATH = 'finnhub_data.db'
SYMBOL = 'MSFT'

## 1. Data Extraction
We query the `v_model_features` view created in Phase 3 (SQL Pipelining). This view already contains our engineered features:
* `sales_growth_yoy`: Year-over-Year growth in Sales Per Share.
* `eps_ma_4q`: 4-Quarter Moving Average of EPS (Smoothing trend).
* `target_next_eps`: The EPS of the *next* quarter (our prediction target).

In [ ]:
conn = sqlite3.connect(DB_PATH)
query = """
SELECT period, eps, target_next_eps, sales_growth_yoy, eps_ma_4q, net_margin, total_debt_to_equity
FROM v_model_features
WHERE symbol = ?
ORDER BY period ASC
"""
df = pd.read_sql_query(query, conn, params=(SYMBOL,))
conn.close()

df['period'] = pd.to_datetime(df['period'])
df_clean = df.dropna()

print(f"Total Records: {len(df)}")
print(f"Records available for ML (after cleaning nulls): {len(df_clean)}")
df_clean.tail()

## 2. Exploratory Data Analysis (EDA)
Visualizing the long-term trend of Earnings Per Share (EPS) and its Moving Average.

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df['period'], df['eps'], label='Actual EPS', alpha=0.5)
plt.plot(df['period'], df['eps_ma_4q'], label='4-Qtr Moving Average', linewidth=2, color='red')
plt.title(f'{SYMBOL} Quarterly EPS Trend')
plt.xlabel('Date')
plt.ylabel('EPS ($)')
plt.legend()
plt.grid(True)
plt.show()

### Feature Correlation
Checking how our features correlate with the target (`target_next_eps`).

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(df_clean[['target_next_eps', 'sales_growth_yoy', 'net_margin', 'total_debt_to_equity', 'eps_ma_4q']].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

## 3. Machine Learning: EPS Forecasting
We use **Linear Regression** to predict the *next quarter's EPS* based on current financial health indicators.

**Features (X):**
1. Sales Growth (YoY)
2. Net Margin
3. Total Debt / Equity
4. Trailing 4-Qtr EPS Average

**Target (Y):** Next Quarter EPS

In [ ]:
features = ['sales_growth_yoy', 'net_margin', 'total_debt_to_equity', 'eps_ma_4q']
target = 'target_next_eps'

X = df_clean[features]
y = df_clean[target]

# Time-Series Split (80% Train, 20% Test)
# We do not shuffle because the data is time-dependent
split_idx = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print(f"Training Samples: {len(X_train)}")
print(f"Testing Samples: {len(X_test)}")

In [ ]:
# Model Training
model = LinearRegression()
model.fit(X_train, y_train)

# Prediction
y_pred = model.predict(X_test)

# Evaluation
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Root Mean Squared Error (RMSE): ${rmse:.4f}")
print(f"R2 Score: {r2:.4f}")

## 4. Alternative Approach: Classification
Since predicting the exact EPS value is difficult due to volatility, we try a simpler task: **Predicting Direction**.
* **Target**: `1` if Next Quarter EPS > Current EPS (Growth), `0` otherwise.
* **Model**: Random Forest Classifier.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Create Binary Target
df_clean['target_direction'] = (df_clean['target_next_eps'] > df_clean['eps']).astype(int)

y_class = df_clean['target_direction']

# Split (Same Time-Series Split)
y_train_c, y_test_c = y_class.iloc[:split_idx], y_class.iloc[split_idx:]

# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train_c)

# Predict
y_pred_c = rf_model.predict(X_test)

# Evaluate
print(f"Accuracy: {accuracy_score(y_test_c, y_pred_c):.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test_c, y_pred_c))
print("\nClassification Report:")
print(classification_report(y_test_c, y_pred_c))

## 5. Conclusion
We applied both Regression and Classification to the quarterly financial data.
* The **Regression model** achieved an RMSE of ~$0.91.
* The **Classification model** highlighted the difficulty of predicting short-term quarterly movements based purely on historical fundamental ratios.

**Future Improvements:**
* Incorporate **Macro-economic data** (Interest Rates, Inflation) which heavily impacts Tech stocks like MSFT.
* Use **Sentiment Analysis** from News (Finnhub 'News' endpoint) to capture market mood.